### Download the dataset about Mnist

In [ ]:
%cd dataset
!wget http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz   #训练集
!wget http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz   #训练集标签
!wget http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz    #测试集
!wget http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz    #测试集标签

#### 导入相关包

In [1]:
import torch
from torch import nn
from torch.autograd.variable import Variable

import torchvision
import torchvision.transforms as transforms

In [ ]:
# Preprocess
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ]
)